In [1]:
import xml.etree.ElementTree as ET
import csv
import os

In [2]:
def convert_xml_to_csv(xml_file, csv_file, element_name):
    """
    Converts a SUMO XML output file to a CSV file, handling nested and inconsistent elements.

    Args:
        xml_file (str): The path to the input XML file.
        csv_file (str): The path to the output CSV file.
        element_name (str): The name of the XML elements to process (e.g., 'tripinfo', 'step', 'edge').
    """
    try:
        # Check if the XML file exists
        if not os.path.exists(xml_file):
            print(f"Error: The file '{xml_file}' was not found.")
            return

        # Parse the XML file
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Use an XPath expression to find all elements with the given tag, regardless of their depth.
        elements_to_process = root.findall(f'.//{element_name}')

        # Check if any elements were found
        if not elements_to_process:
            print(f"No elements with tag '{element_name}' found in '{xml_file}'.")
            return

        # --- MODIFICATION START ---
        # Collect all unique attribute keys from all elements to create a comprehensive header list.
        # This handles cases where elements have different sets of attributes.
        all_headers = set()
        for element in elements_to_process:
            all_headers.update(element.attrib.keys())
        headers = sorted(list(all_headers)) # Sort for consistent column order
        # --- MODIFICATION END ---

        # Open the CSV file for writing
        with open(csv_file, 'w', newline='') as csvfile:
            # Use the comprehensive list of headers
            writer = csv.DictWriter(csvfile, fieldnames=headers)

            # Write the header row
            writer.writeheader()

            # Write the data rows
            for element in elements_to_process:
                writer.writerow(element.attrib)

        print(f"Successfully converted '{xml_file}' to '{csv_file}'")

    except ET.ParseError:
        print(f"Error: Failed to parse '{xml_file}'. Make sure it is a valid XML file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [12]:
if __name__ == "__main__":
    # Define the files to be converted
    # Assumes the XML files are in the same directory as the script
    files_to_convert = [
        {'xml': 'output_tripinfo.xml', 'csv': 'tripinfo.csv', 'element': 'tripinfo'},
        {'xml': 'output_summary.xml', 'csv': 'summary.csv', 'element': 'step'},
        {'xml': 'output_edgedata.xml', 'csv': 'edgedata.csv', 'element': 'edge'}
    ]

    # Convert each file
    for f in files_to_convert:
        convert_xml_to_csv(f['xml'], f['csv'], f['element'])

Successfully converted 'output_tripinfo.xml' to 'tripinfo.csv'
Successfully converted 'output_summary.xml' to 'summary.csv'
Successfully converted 'output_edgedata.xml' to 'edgedata.csv'
